In [52]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import roc_auc_score
%matplotlib inline

Решается задача кредитного скоринга.

Признаки клиентов банка:

    Age - возраст (вещественный)
    Income - месячный доход (вещественный)
    BalanceToCreditLimit - отношение баланса на кредитной карте к лимиту по кредиту (вещественный)
    DIR - Debt-to-income Ratio (вещественный)
    NumLoans - число заемов и кредитных линий
    NumRealEstateLoans - число ипотек и заемов, связанных с недвижимостью (натуральное число)
    NumDependents - число членов семьи, которых содержит клиент, исключая самого клиента (натуральное число)
    Num30-59Delinquencies - число просрочек выплат по кредиту от 30 до 59 дней (натуральное число)
    Num60-89Delinquencies - число просрочек выплат по кредиту от 60 до 89 дней (натуральное число)
    Delinquent90 - были ли просрочки выплат по кредиту более 90 дней (бинарный) - имеется только в обучающей выборке


In [4]:
train_df = pd.read_csv('C:\projects\Python\mlcourse_open\Lesson5\data/credit_scoring_train.csv', index_col='client_id')
test_df = pd.read_csv('C:\projects\Python\mlcourse_open\Lesson5\data/credit_scoring_test.csv', index_col='client_id')

In [6]:
train_df.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit,Delinquent90
client_id,,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028,0
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679,0
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227,0
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930,0
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350,0


In [7]:
y = train_df['Delinquent90']
train_df.drop('Delinquent90', axis=1, inplace=True)

In [8]:
train_df.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit
client_id,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350


In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 0 to 74999
Data columns (total 9 columns):
DIR                      75000 non-null float64
Age                      75000 non-null float64
NumLoans                 75000 non-null int64
NumRealEstateLoans       75000 non-null int64
NumDependents            73084 non-null float64
Num30-59Delinquencies    75000 non-null int64
Num60-89Delinquencies    75000 non-null int64
Income                   60153 non-null float64
BalanceToCreditLimit     75000 non-null float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


In [10]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 75000 to 149999
Data columns (total 9 columns):
DIR                      75000 non-null float64
Age                      75000 non-null float64
NumLoans                 75000 non-null int64
NumRealEstateLoans       75000 non-null int64
NumDependents            72992 non-null float64
Num30-59Delinquencies    75000 non-null int64
Num60-89Delinquencies    75000 non-null int64
Income                   60116 non-null float64
BalanceToCreditLimit     75000 non-null float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


### Заполняем Пропуски средними значениями по своим категориям

In [15]:
train_df['NumDependents'].fillna(train_df['NumDependents'].median(), inplace=True)
train_df['Income'].fillna(train_df['Income'].median(), inplace=True)
test_df['NumDependents'].fillna(test_df['NumDependents'].median(), inplace=True)
test_df['Income'].fillna(test_df['Income'].median(), inplace=True)


### Дерево решений без настройки параметров

Обучите дерево решений максимальной глубины 3, используйте параметр random_state=17 для воспроизводимости результатов.


In [17]:
first_tree = RandomForestClassifier(max_depth=3, random_state=17)
first_tree.fit(train_df, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

### Прогноз для тренировочной выборки 

In [40]:
first_tree_pred = first_tree.predict_proba(train_df)[:,1]
first_tree_pred

array([ 0.09700244,  0.07842017,  0.0782968 , ...,  0.11752608,
        0.03785438,  0.02606583])

In [41]:
score = roc_auc_score(y, first_tree_pred)
score

0.83078507147793779


### Дерево решений с настройкой параметров с помощью GridSearch

Настройте параметры дерева с помощью GridSearhCV, посмотрите на лучшую комбинацию параметров и среднее качество на 5-кратной кросс-валидации. Используйте параметр random_state=17 (для воспроизводимости результатов), не забывайте про распараллеливание (n_jobs=-1).


In [53]:
# Инициализируем страифицированную разбивку нашего датасета для валидации
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [ ]:
tree_params = {'max_depth': list(range(3, 8)), 
               'min_samples_leaf': list(range(5, 13)),
               'max_features': np.linspace(.3, 1, 7),
               'n_estimators': list(range(3,100))}

locally_best_tree = GridSearchCV(first_tree, tree_params, cv=3, n_jobs= -1)
locally_best_tree.fit(train_df, y)

In [55]:
locally_best_tree.best_params_, locally_best_tree.best_score_

({'max_depth': 6, 'max_features': 0.53333333333333333, 'min_samples_leaf': 11},
 0.93530666666666662)

### Второй вариант

In [47]:
tree_params = {'max_features': np.linspace(.3, 1, 7)}

locally_best_tree2 = GridSearchCV(first_tree, tree_params, cv=5, n_jobs= -1)
locally_best_tree2.fit(train_df, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_features': array([ 0.3    ,  0.41667,  0.53333,  0.65   ,  0.76667,  0.88333,  1.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [48]:
locally_best_tree2.best_params_, locally_best_tree2.best_score_

({'max_features': 1.0}, 0.93465333333333334)

In [ ]:
pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_

### Параметры

Метод случайного леса реализован в библиотеке машинного обучения scikit-learn двумя классами RandomForestClassifier и RandomForestRegressor.

Полный список параметров случайного леса для задачи регрессии:

class sklearn.ensemble.RandomForestRegressor(
    n_estimators — число деревьев в "лесу" (по дефолту – 10)
    
    criterion — функция, которая измеряет качество разбиения ветки дерева (по дефолту — "mse" , так же можно выбрать "mae")
    
    max_features — число признаков, по которым ищется разбиение. Вы можете указать конкретное число или процент признаков, либо выбрать из доступных значений: "auto" (все признаки), "sqrt", "log2". По дефолту стоит "auto".
    
    max_depth — максимальная глубина дерева  (по дефолту глубина не ограничена)
    
    min_samples_split — минимальное количество объектов, необходимое для разделения внутреннего узла. Можно задать числом или процентом от общего числа объектов (по дефолту — 2)
    
    min_samples_leaf — минимальное число объектов в листе. Можно задать числом или процентом от общего числа объектов (по дефолту — 1)
    
    min_weight_fraction_leaf — минимальная взвешенная доля от общей суммы весов (всех входных объектов) должна быть в листе (по дефолту имеют одинаковый вес)
    
    max_leaf_nodes — максимальное количество листьев (по дефолту нет ограничения)
    
    min_impurity_split — порог для остановки наращивания дерева (по дефолту 1е-7)
   
    bootstrap — применять ли бустрэп для построения дерева (по дефолту True)
    
    oob_score — использовать ли out-of-bag объекты для оценки R^2 (по дефолту False)
   
    n_jobs — количество ядер для построения модели и предсказаний (по дефолту 1, если поставить -1, то будут использоваться все ядра)
    
    random_state — начальное значение для генерации случайных чисел (по дефолту его нет, если хотите воспроизводимые результаты, то нужно указать любое число типа int
    
    verbose — вывод логов по построению деревьев (по дефолту 0)
    
    warm_start — использует уже натренированую модель и добавляет деревьев в ансамбль (по дефолту False)
)

Для задачи классификации все почти то же самое, мы приведем только те параметры, которыми RandomForestClassifier отличается от RandomForestRegressor

class sklearn.ensemble.RandomForestClassifier(
    
    criterion — поскольку у нас теперь задача классификации, то по дефолту выбран критерий "gini" (можно выбрать "entropy")
    
    class_weight — вес каждого класса (по дефолту все веса равны 1, но можно передать словарь с весами, либо явно указать "balanced", тогда веса классов будут равны их исходным частям в генеральной совокупности; также можно указать "balanced_subsample", тогда веса на каждой подвыборке будут меняться в зависимости от распределения классов на этой подвыборке.

Далее рассмотрим несколько параметров, на которые в первую очередь стоит обратить внимание при построении модели:

    n_estimators — число деревьев в "лесу"
    criterion — критерий для разбиения выборки в вершине
    max_features — число признаков, по которым ищется разбиение
    min_samples_leaf — минимальное число объектов в листе
    max_depth — максимальная глубина дерева
